In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data121036  data126186


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

# 任务
对比赛给定的带有摩尔纹的图片进行处理，消除屏摄产生的摩尔纹噪声，还原图片原本的样子
# 评价指标
![](https://ai-studio-static-online.cdn.bcebos.com/47963fc72fb84278a46c51b2e852dd15fadd9b15b14f4e5ba21b5c9bc2690f03)

# 数据
训练集1000张，测试集A、B各200张

![](https://ai-studio-static-online.cdn.bcebos.com/b561143cbef843c2ae94251d8a1e8d97ad3e7714d06242d88b828aec459ba381)

# 难点
1、摩尔纹形态有多种，没有统一的特征

2、摩尔纹在整张图上分布不均匀
![](https://ai-studio-static-online.cdn.bcebos.com/d14f646643864c37821a67c58f26a702bd7cc1a91a75430d851e3c1f522e0ae1)

# 数据处理
1000张训练数据使用900张训练，100张验证
# 模型实现
![](https://ai-studio-static-online.cdn.bcebos.com/1116e0f0f175465c8378b7609c770423e9795099225e459e8ec4dd3ac829169d)

Unet,降采样4次，使用MaxPool,升采样4次，使用ConvTranspose,层间有跨连接

优点：参数量少、速度快、对均匀分布的摩尔纹效果好

缺点：对于非均匀分布的摩尔纹，处理效果不佳

![](https://ai-studio-static-online.cdn.bcebos.com/f46c99639b4d407995c28a269f673e990d344ac4f0cf41a0b1b5628cc63ac1fa)

# 训练
1、训练时数据增强：裁剪为512x512大小的patch进行训练，以0.5的概率随机旋转、翻转

2、损失函数:直接使用评分指标做为loss

3、学习率：使用stepLR，从1e-4降低到1e-5

4、优化器：Adam

# 测试细节
数据增强：将原图旋转、翻转后的预测结果取平均，PSNR可提升0.36db

# 主要提分点
1、unet结构中使用maxpool作为下采样

2、使用score作为loss

3、使用大patch进行训练

4、测试增强



In [16]:
# 解压数据
!unzip data/data121036/moire_train_dataset.zip -d data/train >/dev/null
# !unzip data/data121036/moire_testA_dataset.zip -d data/testA >/dev/null
!unzip data/data126186/moire_testB_dataset.zip -d data/testB >/dev/null

replace data/train/__MACOSX/._moire_train_dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
replace data/testB/__MACOSX/._moire_testB_dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [15]:
# 启动训练， 模型存放在ckpts
%cd /home/aistudio/
!python work/train.py

/home/aistudio
W0124 11:48:17.504961  4996 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0124 11:48:17.509470  4996 device_context.cc:465] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
epoch: 0 iter: 100 loss: -0.5032016345858574 lr: 0.0001 net: ckpts/sid
epoch: 0 iter: 200 loss: -0.5373259426653385 lr: 0.0001 net: ckpts/sid
epoch: 0 iter: 300 loss: -0.5506085867683093 lr: 0.0001 net: ckpts/sid
epoch: 0 iter: 400 loss: -0.558398531228304 lr: 0.0001 net: ckpts/sid
epoch: 0 iter: 450 ave_psnr: 20

In [ ]:
# 复现B榜，结果存放在res
%cd /home/aistudio/
!python work/test.py

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 